In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torch import nn
import random
from transformers import BertTokenizer

class imbd_dataset(Dataset):
    def __init__(self, file_path, tokenizer):
        df = pd.read_csv(file_path)
        df["sentiment"] = np.where(df["sentiment"] == "positive", 1, 0)
        self.x = df["review"].values
        self.y = df["sentiment"].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        sentence = self.x[idx]
        tokens = self.tokenizer.encode(sentence, max_length=512, truncation=True, padding='max_length')
        tokens_tensor = torch.tensor(tokens)
        y_logs = nn.functional.one_hot(torch.tensor(self.y[idx]), num_classes=2).long()

        return tokens_tensor, y_logs

def create_train_test_datasets(filepath, tokenizer, split_ratio=0.8):
    dataset = imbd_dataset(filepath, tokenizer)
    train_size = int(split_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_data, test_data = random_split(dataset, [train_size, test_size])

    return train_data, test_data

def create_dataloader(training_data, test_data, batch_size):
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

def select_device():
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(device)
    return device

class NeuralNetwork(nn.Module):
    def __init__(self, vocab_size, embed_size=50, hidden_size=100, encoders=1, head=10, num_classes=2):
        super(NeuralNetwork, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(embed_size, hidden_size, num_layers=20, bidirectional=True)
        self.h0 = torch.zeros(20 * num_classes, 512, hidden_size)
        self.c0 = torch.zeros(20 * num_classes, 512, hidden_size)
        self.last_layer = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size*2, num_classes)
        )

    def forward(self, x):
        embed = self.embed(x)
        out_encod, (hn, cn) = self.encoder(embed, (self.h0, self.c0))
        logits = self.last_layer(out_encod)
        return logits

def load_model(weights, device):
    model = NeuralNetwork(tokenizer.vocab_size).to(device)
    if os.path.exists(weights):
        try:
            model.load_state_dict(torch.load(weights), strict=False)
        except Exception as e:
            print(f"None weights were loaded: {e}")
    return model

def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 25 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    global best_weights, best_correct
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    if best_correct < correct:
        best_correct = correct
        best_weights = model.state_dict()


def __main__(file_path, learning_rate, batch_size, epochs):
    loss_fn = nn.CrossEntropyLoss()
    device = select_device()
    model = load_model('model_weights.pth', device)
    train_on = input("Do you want to train? Yes(y), No(enter): ")
    if train_on:
        training_data, test_data = create_train_test_datasets(file_path, tokenizer)
        train_dataloader, test_dataloader = create_dataloader(training_data, test_data, batch_size)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        next_train = True
        while next_train:
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loop(train_dataloader, model, loss_fn, optimizer, device)
                test_loop(test_dataloader, model, loss_fn, device)
            print("Done! Saving process...")
            torch.save(model.state_dict(), 'model_weights.pth')
            print(f"Best Accuracy: {(100*best_correct):>0.1f}% Using Learning Rate: {learning_rate}")
            stop_train = input("Should we continue training? No(Any) Yes(Enter) -> ")
            if stop_train:
                next_train = False
            else:
                change_lr = input("Change the learning rate? Yes(Any) No(Enter) -> ")
                if change_lr:
                    try:
                        learning_rate = float(input("New learning rate: "))
                    except:
                        print("Ignoring new learning rate")
                change_epochs = input(f"Change the number of epochs ({epochs})? Yes(Any) No(Enter) -> ")
                if change_epochs:
                    try:
                        epochs = int(input("New number of epochs: "))
                    except:
                        print("Ignoring new number of epochs")
    else:
        training_data, test_data = create_train_test_datasets(file_path, tokenizer)
        _, test_dataloader = create_dataloader(training_data, test_data, batch_size)
        test_loop(test_dataloader, model, loss_fn, device)

if __name__ == '__main__':
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    best_correct = 0
    best_weights = None
    file_path = 'IMDB_Dataset.csv'
    learning_rate = 0.0008
    batch_size = 10
    epochs = 10
    __main__(file_path, learning_rate, batch_size, epochs)

cuda


Do you want to train? Yes(y), No(enter):  y


Epoch 1
-------------------------------


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu